<img src="images/dask_horizontal.svg" align="right" width="30%">

# Bag: Parallel Lists for semi-structured data

Dask-bag excels in processing data that can be represented as a sequence of arbitrary inputs. We'll refer to this as "messy" data, because it can contain complex nested structures, missing fields, mixtures of data types, etc. The *functional* programming style fits very nicely with standard Python iteration, such as can be found in the `itertools` module.

Messy data is often encountered at the beginning of data processing pipelines when large volumes of raw data are first consumed. The initial set of data might be JSON, CSV, XML, or any other format that does not enforce strict structure and datatypes.
For this reason, the initial data massaging and processing is often done with Python `list`s, `dict`s, and `set`s.

These core data structures are optimized for general-purpose storage and processing.  Adding streaming computation with iterators/generator expressions or libraries like `itertools` or [`toolz`](https://toolz.readthedocs.io/en/latest/) let us process large volumes in a small space.  If we combine this with parallel processing then we can churn through a fair amount of data.

Dask.bag is a high level Dask collection to automate common workloads of this form.  In a nutshell

    dask.bag = map, filter, toolz + parallel execution
    
**Related Documentation**

* [Bag documentation](https://docs.dask.org/en/latest/bag.html)
* [Bag screencast](https://youtu.be/-qIiJ1XtSv0)
* [Bag API](https://docs.dask.org/en/latest/bag-api.html)
* [Bag examples](https://examples.dask.org/bag.html)

## Create data

In [1]:
%run prep.py -d accounts

Created CSV accounts in 7.26s
Created JSON accounts in 48.62s


## Setup

Again, we'll use the distributed scheduler. Schedulers will be explained in depth [later](05_distributed.ipynb).

In [2]:
from dask.distributed import Client

client = Client(n_workers=4)

## Creation

You can create a `Bag` from a Python sequence, from files, from data on S3, etc.
We demonstrate using `.take()` to show elements of the data. (Doing `.take(1)` results in a tuple with one element)

Note that the data are partitioned into blocks, and there are many items per block. In the first example, the two partitions contain five elements each, and in the following two, each file is partitioned into one or more bytes blocks.

In [3]:
# each element is an integer
import dask.bag as db
b = db.from_sequence([1, 2, 3, 4, 5, 6, 7, 8, 9, 10], npartitions=2)
b.take(3)

(1, 2, 3)

In [4]:
# each element is a text file, where each line is a JSON object
# note that the compression is handled automatically
import os
b = db.read_text(os.path.join('data', 'accounts.*.json.gz'))
b.take(1)

('{"id": 0, "name": "Bob", "transactions": [{"transaction-id": 3850, "amount": 6116}, {"transaction-id": 4611, "amount": 6567}, {"transaction-id": 14422, "amount": 6245}, {"transaction-id": 19148, "amount": 6646}, {"transaction-id": 20449, "amount": 6486}, {"transaction-id": 21205, "amount": 6317}, {"transaction-id": 21431, "amount": 6717}, {"transaction-id": 22600, "amount": 6485}, {"transaction-id": 24554, "amount": 6627}, {"transaction-id": 25814, "amount": 6260}, {"transaction-id": 26369, "amount": 6312}, {"transaction-id": 43303, "amount": 6366}, {"transaction-id": 46604, "amount": 6581}, {"transaction-id": 47882, "amount": 6382}, {"transaction-id": 50301, "amount": 6418}, {"transaction-id": 52695, "amount": 6522}, {"transaction-id": 59206, "amount": 6480}, {"transaction-id": 71276, "amount": 6619}, {"transaction-id": 71704, "amount": 6507}, {"transaction-id": 73398, "amount": 6302}, {"transaction-id": 74028, "amount": 5930}, {"transaction-id": 76718, "amount": 6454}, {"transactio

In [5]:
# Edit sources.py to configure source locations
import sources
sources.bag_url

's3://dask-data/nyc-taxi/2015/yellow_tripdata_2015-01.csv'

In [6]:
# Requires `s3fs` library
# each partition is a remote CSV text file
b = db.read_text(sources.bag_url,
                 storage_options={'anon': True})
b.take(1)

('VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,pickup_longitude,pickup_latitude,RateCodeID,store_and_fwd_flag,dropoff_longitude,dropoff_latitude,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount\n',)

## Manipulation

`Bag` objects hold the standard functional API found in projects like the Python standard library, `toolz`, or `pyspark`, including `map`, `filter`, `groupby`, etc..

Operations on `Bag` objects create new bags.  Call the `.compute()` method to trigger execution, as we saw for `Delayed` objects.  

In [7]:
def is_even(n):
    return n % 2 == 0

b = db.from_sequence([1, 2, 3, 4, 5, 6, 7, 8, 9, 10])
c = b.filter(is_even).map(lambda x: x ** 2)
c

dask.bag<lambda, npartitions=10>

In [8]:
# blocking form: wait for completion (which is very fast in this case)
c.compute()

[4, 16, 36, 64, 100]

### Example: Accounts JSON data

We've created a fake dataset of gzipped JSON data in your data directory.  This is like the example used in the `DataFrame` example we will see later, except that it has bundled up all of the entries for each individual `id` into a single record.  This is similar to data that you might collect off of a document store database or a web API.

Each line is a JSON encoded dictionary with the following keys

*  id: Unique identifier of the customer
*  name: Name of the customer
*  transactions: List of `transaction-id`, `amount` pairs, one for each transaction for the customer in that file

In [9]:
filename = os.path.join('data', 'accounts.*.json.gz')
lines = db.read_text(filename)
lines.take(3)

('{"id": 0, "name": "Bob", "transactions": [{"transaction-id": 3850, "amount": 6116}, {"transaction-id": 4611, "amount": 6567}, {"transaction-id": 14422, "amount": 6245}, {"transaction-id": 19148, "amount": 6646}, {"transaction-id": 20449, "amount": 6486}, {"transaction-id": 21205, "amount": 6317}, {"transaction-id": 21431, "amount": 6717}, {"transaction-id": 22600, "amount": 6485}, {"transaction-id": 24554, "amount": 6627}, {"transaction-id": 25814, "amount": 6260}, {"transaction-id": 26369, "amount": 6312}, {"transaction-id": 43303, "amount": 6366}, {"transaction-id": 46604, "amount": 6581}, {"transaction-id": 47882, "amount": 6382}, {"transaction-id": 50301, "amount": 6418}, {"transaction-id": 52695, "amount": 6522}, {"transaction-id": 59206, "amount": 6480}, {"transaction-id": 71276, "amount": 6619}, {"transaction-id": 71704, "amount": 6507}, {"transaction-id": 73398, "amount": 6302}, {"transaction-id": 74028, "amount": 5930}, {"transaction-id": 76718, "amount": 6454}, {"transactio

Our data comes out of the file as lines of text. Notice that file decompression happened automatically. We can make this data look more reasonable by mapping the `json.loads` function onto our bag.

In [10]:
import json
js = lines.map(json.loads)
# take: inspect first few elements
js.take(3)

({'id': 0,
  'name': 'Bob',
  'transactions': [{'transaction-id': 3850, 'amount': 6116},
   {'transaction-id': 4611, 'amount': 6567},
   {'transaction-id': 14422, 'amount': 6245},
   {'transaction-id': 19148, 'amount': 6646},
   {'transaction-id': 20449, 'amount': 6486},
   {'transaction-id': 21205, 'amount': 6317},
   {'transaction-id': 21431, 'amount': 6717},
   {'transaction-id': 22600, 'amount': 6485},
   {'transaction-id': 24554, 'amount': 6627},
   {'transaction-id': 25814, 'amount': 6260},
   {'transaction-id': 26369, 'amount': 6312},
   {'transaction-id': 43303, 'amount': 6366},
   {'transaction-id': 46604, 'amount': 6581},
   {'transaction-id': 47882, 'amount': 6382},
   {'transaction-id': 50301, 'amount': 6418},
   {'transaction-id': 52695, 'amount': 6522},
   {'transaction-id': 59206, 'amount': 6480},
   {'transaction-id': 71276, 'amount': 6619},
   {'transaction-id': 71704, 'amount': 6507},
   {'transaction-id': 73398, 'amount': 6302},
   {'transaction-id': 74028, 'amount':

### Basic Queries

Once we parse our JSON data into proper Python objects (`dict`s, `list`s, etc.) we can perform more interesting queries by creating small Python functions to run on our data.

In [11]:
# filter: keep only some elements of the sequence
js.filter(lambda record: record['name'] == 'Alice').take(5)

({'id': 19,
  'name': 'Alice',
  'transactions': [{'transaction-id': 617, 'amount': 243},
   {'transaction-id': 1692, 'amount': 139},
   {'transaction-id': 2823, 'amount': 213},
   {'transaction-id': 2984, 'amount': 167},
   {'transaction-id': 3779, 'amount': 221},
   {'transaction-id': 4091, 'amount': 159},
   {'transaction-id': 4172, 'amount': 146},
   {'transaction-id': 4553, 'amount': 210},
   {'transaction-id': 6310, 'amount': 207},
   {'transaction-id': 6746, 'amount': 138},
   {'transaction-id': 7151, 'amount': 167},
   {'transaction-id': 8202, 'amount': 176},
   {'transaction-id': 8365, 'amount': 228},
   {'transaction-id': 8612, 'amount': 205},
   {'transaction-id': 9107, 'amount': 172},
   {'transaction-id': 9582, 'amount': 128},
   {'transaction-id': 9963, 'amount': 158},
   {'transaction-id': 10536, 'amount': 184},
   {'transaction-id': 10573, 'amount': 159},
   {'transaction-id': 10839, 'amount': 174},
   {'transaction-id': 10866, 'amount': 221},
   {'transaction-id': 1097

In [12]:
def count_transactions(d):
    return {'name': d['name'], 'count': len(d['transactions'])}

# map: apply a function to each element
(js.filter(lambda record: record['name'] == 'Alice')
   .map(count_transactions)
   .take(5))

({'name': 'Alice', 'count': 198},
 {'name': 'Alice', 'count': 93},
 {'name': 'Alice', 'count': 244},
 {'name': 'Alice', 'count': 100},
 {'name': 'Alice', 'count': 271})

In [13]:
# pluck: select a field, as from a dictionary, element[field]
(js.filter(lambda record: record['name'] == 'Alice')
   .map(count_transactions)
   .pluck('count')
   .take(5))

(198, 93, 244, 100, 271)

In [14]:
# Average number of transactions for all of the Alice entries
(js.filter(lambda record: record['name'] == 'Alice')
   .map(count_transactions)
   .pluck('count')
   .mean()
   .compute())

161.452

### Use `flatten` to de-nest

In the example below we see the use of `.flatten()` to flatten results.  We compute the average amount for all transactions for all Alices.

In [15]:
(js.filter(lambda record: record['name'] == 'Alice')
   .pluck('transactions')
   .take(3))

([{'transaction-id': 617, 'amount': 243},
  {'transaction-id': 1692, 'amount': 139},
  {'transaction-id': 2823, 'amount': 213},
  {'transaction-id': 2984, 'amount': 167},
  {'transaction-id': 3779, 'amount': 221},
  {'transaction-id': 4091, 'amount': 159},
  {'transaction-id': 4172, 'amount': 146},
  {'transaction-id': 4553, 'amount': 210},
  {'transaction-id': 6310, 'amount': 207},
  {'transaction-id': 6746, 'amount': 138},
  {'transaction-id': 7151, 'amount': 167},
  {'transaction-id': 8202, 'amount': 176},
  {'transaction-id': 8365, 'amount': 228},
  {'transaction-id': 8612, 'amount': 205},
  {'transaction-id': 9107, 'amount': 172},
  {'transaction-id': 9582, 'amount': 128},
  {'transaction-id': 9963, 'amount': 158},
  {'transaction-id': 10536, 'amount': 184},
  {'transaction-id': 10573, 'amount': 159},
  {'transaction-id': 10839, 'amount': 174},
  {'transaction-id': 10866, 'amount': 221},
  {'transaction-id': 10975, 'amount': 141},
  {'transaction-id': 11350, 'amount': 171},
  {'tr

In [16]:
(js.filter(lambda record: record['name'] == 'Alice')
   .pluck('transactions')
   .flatten()
   .take(3))

({'transaction-id': 617, 'amount': 243},
 {'transaction-id': 1692, 'amount': 139},
 {'transaction-id': 2823, 'amount': 213})

In [17]:
(js.filter(lambda record: record['name'] == 'Alice')
   .pluck('transactions')
   .flatten()
   .pluck('amount')
   .take(3))

(243, 139, 213)

In [18]:
(js.filter(lambda record: record['name'] == 'Alice')
   .pluck('transactions')
   .flatten()
   .pluck('amount')
   .mean()
   .compute())

599.429237998497

### Groupby and Foldby

Often we want to group data by some function or key.  We can do this either with the `.groupby` method, which is straightforward but forces a full shuffle of the data (expensive) or with the harder-to-use but faster `.foldby` method, which does a streaming combined groupby and reduction.

*  `groupby`:  Shuffles data so that all items with the same key are in the same key-value pair
*  `foldby`:  Walks through the data accumulating a result per key

*Note: the full groupby is particularly bad. In actual workloads you would do well to use `foldby` or switch to `DataFrame`s if possible.*

### `groupby`

Groupby collects items in your collection so that all items with the same value under some function are collected together into a key-value pair.

In [19]:
b = db.from_sequence(['Alice', 'Bob', 'Charlie', 'Dan', 'Edith', 'Frank'])
b.groupby(len).compute()  # names grouped by length

[(7, ['Charlie']), (3, ['Bob', 'Dan']), (5, ['Alice', 'Edith', 'Frank'])]

In [20]:
b = db.from_sequence(list(range(10)))
b.groupby(lambda x: x % 2).compute()

[(0, [0, 2, 4, 6, 8]), (1, [1, 3, 5, 7, 9])]

In [21]:
b.groupby(lambda x: x % 2).starmap(lambda k, v: (k, max(v))).compute()

[(0, 8), (1, 9)]

### `foldby`

Foldby can be quite odd at first.  It is similar to the following functions from other libraries:

*  [`toolz.reduceby`](http://toolz.readthedocs.io/en/latest/streaming-analytics.html#streaming-split-apply-combine)
*  [`pyspark.RDD.combineByKey`](http://abshinn.github.io/python/apache-spark/2014/10/11/using-combinebykey-in-apache-spark/)

When using `foldby` you provide 

1.  A key function on which to group elements
2.  A binary operator such as you would pass to `reduce` that you use to perform reduction per each group
3.  A combine binary operator that can combine the results of two `reduce` calls on different parts of your dataset.

Your reduction must be associative.  It will happen in parallel in each of the partitions of your dataset.  Then all of these intermediate results will be combined by the `combine` binary operator.

In [22]:
b.foldby(lambda x: x % 2, binop=max, combine=max).compute()

[(0, 8), (1, 9)]

### Example with account data

We find the number of people with the same name.

In [23]:
%%time
# Warning, this one takes a while...
result = js.groupby(lambda item: item['name']).starmap(lambda k, v: (k, len(v))).compute()
print(sorted(result))

[('Alice', 180), ('Alice', 180), ('Alice', 180), ('Alice', 210), ('Bob', 264), ('Bob', 264), ('Bob', 264), ('Bob', 308), ('Charlie', 168), ('Charlie', 168), ('Charlie', 168), ('Charlie', 196), ('Dan', 180), ('Dan', 180), ('Dan', 180), ('Dan', 210), ('Edith', 252), ('Edith', 252), ('Edith', 252), ('Edith', 294), ('Frank', 275), ('Frank', 276), ('Frank', 276), ('Frank', 322), ('George', 252), ('George', 252), ('George', 252), ('George', 294), ('Hannah', 225), ('Hannah', 225), ('Hannah', 227), ('Hannah', 266), ('Ingrid', 336), ('Ingrid', 336), ('Ingrid', 336), ('Ingrid', 392), ('Jerry', 264), ('Jerry', 264), ('Jerry', 572), ('Kevin', 192), ('Kevin', 192), ('Kevin', 192), ('Kevin', 224), ('Laura', 192), ('Laura', 192), ('Laura', 192), ('Laura', 224), ('Michael', 215), ('Michael', 216), ('Michael', 216), ('Michael', 252), ('Norbert', 222), ('Norbert', 224), ('Norbert', 225), ('Norbert', 261), ('Oliver', 216), ('Oliver', 216), ('Oliver', 216), ('Oliver', 252), ('Patricia', 180), ('Patricia',

In [24]:
%%time
# This one is comparatively fast and produces the same result.
from operator import add
def incr(tot, _):
    return tot + 1

result = js.foldby(key='name', 
                   binop=incr, 
                   initial=0, 
                   combine=add, 
                   combine_initial=0).compute()
print(sorted(result))

[('Alice', 750), ('Bob', 1100), ('Charlie', 700), ('Dan', 750), ('Edith', 1050), ('Frank', 1149), ('George', 1050), ('Hannah', 943), ('Ingrid', 1400), ('Jerry', 1100), ('Kevin', 800), ('Laura', 800), ('Michael', 899), ('Norbert', 932), ('Oliver', 900), ('Patricia', 750), ('Quinn', 1050), ('Ray', 1282), ('Sarah', 734), ('Tim', 1200), ('Ursula', 841), ('Victor', 798), ('Wendy', 850), ('Xavier', 849), ('Yvonne', 1493), ('Zelda', 750)]
CPU times: total: 469 ms
Wall time: 3.49 s


### Exercise: compute total amount per name

We want to groupby (or foldby) the `name` key, then add up the all of the amounts for each name.

Steps

1.  Create a small function that, given a dictionary like 

        {'name': 'Alice', 'transactions': [{'amount': 1, 'id': 123}, {'amount': 2, 'id': 456}]}
        
    produces the sum of the amounts, e.g. `3`
    
2.  Slightly change the binary operator of the `foldby` example above so that the binary operator doesn't count the number of entries, but instead accumulates the sum of the amounts.

In [25]:
# Your code here...

## DataFrames

For the same reasons that Pandas is often faster than pure Python, `dask.dataframe` can be faster than `dask.bag`.  We will work more with DataFrames later, but from the point of view of a Bag, it is frequently the end-point of the "messy" part of data ingestion—once the data can be made into a data-frame, then complex split-apply-combine logic will become much more straight-forward and efficient.

You can transform a bag with a simple tuple or flat dictionary structure into a `dask.dataframe` with the `to_dataframe` method.

In [26]:
df1 = js.to_dataframe()
df1.head()

C:\Users\Rahul\anaconda3\envs\dask-tutorial\lib\site-packages\dask\dataframe\backends.py:187: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  _numeric_index_types = (pd.Int64Index, pd.Float64Index, pd.UInt64Index)
C:\Users\Rahul\anaconda3\envs\dask-tutorial\lib\site-packages\dask\dataframe\backends.py:187: FutureWarning: pandas.Float64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  _numeric_index_types = (pd.Int64Index, pd.Float64Index, pd.UInt64Index)
C:\Users\Rahul\anaconda3\envs\dask-tutorial\lib\site-packages\dask\dataframe\backends.py:187: FutureWarning: pandas.UInt64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  _numeric_index_types = (pd.Int64Index, pd.Float64Index, pd.UInt64Index)


,id,name,transactions
0,0,Bob,"[{'transaction-id': 3850, 'amount': 6116}, {'t..."
1,1,Jerry,"[{'transaction-id': 790, 'amount': -74}, {'tra..."
2,2,Norbert,"[{'transaction-id': 9654, 'amount': 1233}, {'t..."
3,3,Charlie,"[{'transaction-id': 2223, 'amount': -153}, {'t..."
4,4,Hannah,"[{'transaction-id': 194, 'amount': -1}, {'tran..."


This now looks like a well-defined DataFrame, and we can apply Pandas-like computations to it efficiently.

Using a Dask DataFrame, how long does it take to do our prior computation of numbers of people with the same name?  It turns out that `dask.dataframe.groupby()` beats `dask.bag.groupby()` by more than an order of magnitude; but it still cannot match `dask.bag.foldby()` for this case.

In [27]:
%time df1.groupby('name').id.count().compute().head()

CPU times: total: 1.08 s
Wall time: 4.77 s


name
Alice       750
Bob        1100
Charlie     700
Dan         750
Edith      1050
Name: id, dtype: int64

### Denormalization

This DataFrame format is less-than-optimal because the `transactions` column is filled with nested data so Pandas has to revert to `object` dtype, which is quite slow in Pandas.  Ideally we want to transform to a dataframe only after we have flattened our data so that each record is a single `int`, `string`, `float`, etc..

In [28]:
def denormalize(record):
    # returns a list for each person, one item per transaction
    return [{'id': record['id'], 
             'name': record['name'], 
             'amount': transaction['amount'], 
             'transaction-id': transaction['transaction-id']}
            for transaction in record['transactions']]

transactions = js.map(denormalize).flatten()
transactions.take(3)

({'id': 0, 'name': 'Bob', 'amount': 6116, 'transaction-id': 3850},
 {'id': 0, 'name': 'Bob', 'amount': 6567, 'transaction-id': 4611},
 {'id': 0, 'name': 'Bob', 'amount': 6245, 'transaction-id': 14422})

In [29]:
df = transactions.to_dataframe()
df.head()

,id,name,amount,transaction-id
0,0,Bob,6116,3850
1,0,Bob,6567,4611
2,0,Bob,6245,14422
3,0,Bob,6646,19148
4,0,Bob,6486,20449


In [30]:
%%time
# number of transactions per name
# note that the time here includes the data load and ingestion
df.groupby('name')['transaction-id'].count().compute()

CPU times: total: 1.42 s
Wall time: 9.72 s


name
Alice       121089
Bob         192822
Charlie     138240
Dan         205063
Edith       197746
Frank       307999
George      176381
Hannah      126137
Ingrid      288984
Jerry       311628
Kevin       154889
Laura       186447
Michael     150735
Norbert     197522
Oliver      127010
Patricia    157034
Quinn       273092
Ray         226982
Sarah        67775
Tim         187798
Ursula      213515
Victor      205700
Wendy       126773
Xavier      196268
Yvonne      285568
Zelda       176803
Name: transaction-id, dtype: int64

## Limitations

Bags provide very general computation (any Python function.)  This generality
comes at cost.  Bags have the following known limitations

1.  Bag operations tend to be slower than array/dataframe computations in the
    same way that Python tends to be slower than NumPy/Pandas
2.  ``Bag.groupby`` is slow.  You should try to use ``Bag.foldby`` if possible.
    Using ``Bag.foldby`` requires more thought. Even better, consider creating
    a normalised dataframe.

## Learn More

* [Bag documentation](https://docs.dask.org/en/latest/bag.html)
* [Bag screencast](https://youtu.be/-qIiJ1XtSv0)
* [Bag API](https://docs.dask.org/en/latest/bag-api.html)
* [Bag examples](https://examples.dask.org/bag.html)

## Shutdown

In [31]:
client.shutdown()

distributed.client - ERROR - Failed to reconnect to scheduler after 30.00 seconds, closing client
_GatheringFuture exception was never retrieved
future: <_GatheringFuture finished exception=CancelledError()>
asyncio.exceptions.CancelledError
